In [144]:
# Necessary imports
%load_ext autoreload
%autoreload 2\
    
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from definitions import ROOT_DIR



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [145]:
import pandas as pd
import numpy as np

In [146]:
from src.features.multi_omics import MultiOmicsData

lusc_data = MultiOmicsData(cancer_type="LUSC", 
                           folder_path="/home/jonny2/PycharmProjects/assn-miRNA-LUAD/data/tcga-assembler/LUSC/", 
                           modalities=["GE", "MIR"])

('DRUGS', (357, 4))
('PATIENTS', (504, 5))
('MIR', (380, 1870))
('GE', (552, 20472))


In [147]:
centroids = pd.read_csv("/home/jonny2/PycharmProjects/assn-miRNA-LUAD/data/external/wilkerson.scc/predictor.centroids.csv")

In [148]:
centroids.columns = ["genes", "primitive", "classical", "secretory", "basal"]

In [149]:
centroids = centroids[centroids["genes"].isin(lusc_data.GE.get_genes_list())]
genes = centroids["genes"]

In [150]:
centroids.index = centroids.genes
centroids.drop(['genes'], axis=1, inplace=True)
centroids = centroids.T
centroids

genes,MYL6B,PODXL2,HSF2,TTLL4,MARCKSL1,MDK,CHKA,TRIM28,STOM,CASP1,...,ALDH1A3,DSE,MMP10,VDR,CAPZB,FNBP1,ENPP4,SH2B3,DOCK10,SDC1
primitive,0.539568,0.852272,0.293831,0.679557,1.015985,1.001421,0.513166,0.569403,-0.737997,-0.775559,...,-0.231312,-0.521502,-0.862938,-0.421701,-0.015213,-0.016889,0.232617,-0.178687,-0.121821,-0.764157
classical,-0.139755,-0.019997,0.060404,-0.089302,-0.306555,-0.019797,-0.150269,0.021242,-0.065057,-0.187048,...,-0.485903,-0.462036,-0.649346,-0.259443,-0.216407,-0.103185,-0.276022,-0.169656,-0.137791,0.337130
secretory,-0.084411,-0.104802,-0.131566,-0.044189,0.012595,-0.202898,0.235328,-0.264212,0.297794,0.305713,...,0.325191,0.487990,-0.974158,0.384095,0.180478,0.421356,0.482619,0.533466,0.569353,-0.998660
basal,-0.033551,-0.053003,-0.079130,-0.084657,0.097278,-0.222180,-0.136313,0.011287,0.043434,0.063549,...,0.905489,0.509466,2.926716,0.309637,0.216626,-0.185309,-0.147701,-0.030816,-0.093761,0.419894


# Subsetting the GE data to only genes

In [151]:
df = lusc_data.GE.data[genes]

In [131]:
# Normalize 
df_norm = (df.T - df.T.mean()) / (df.T.max() - df.T.min())
df_norm = df_norm.T

In [152]:
df

GeneSymbol,MYL6B,PODXL2,HSF2,TTLL4,MARCKSL1,MDK,CHKA,TRIM28,STOM,CASP1,...,ALDH1A3,DSE,MMP10,VDR,CAPZB,FNBP1,ENPP4,SH2B3,DOCK10,SDC1
TCGA-18-3406-01A,8.577816,8.872572,9.909338,7.645389,10.889552,11.300051,7.075301,12.672723,12.668541,10.408614,...,10.266713,10.538517,9.522867,9.547365,12.263720,9.155555,9.131452,8.979960,8.152105,15.571850
TCGA-18-3407-01A,8.640940,6.962222,8.232251,8.848742,11.501435,10.705123,7.463909,12.333259,12.036452,9.341973,...,9.199516,9.774615,6.068832,10.004453,12.259728,9.583859,7.436436,8.853201,7.670048,14.571843
TCGA-18-3408-01A,8.618790,7.419965,9.128772,8.920247,10.714564,10.636072,7.663135,10.750776,13.232512,9.802814,...,8.110968,10.168276,9.411121,7.917923,12.114407,9.995377,7.882095,8.477076,7.966634,13.587342
TCGA-18-3409-01A,10.097188,9.802185,9.071256,8.908354,12.745066,11.111654,8.387355,12.486957,12.808010,9.664926,...,9.039116,10.733832,5.830291,11.433652,12.494670,9.599674,8.289073,9.432006,8.367331,13.998697
TCGA-18-3410-01A,9.624187,10.405947,9.387628,10.518698,13.486097,12.350812,8.190230,13.420596,11.947963,9.774494,...,10.226253,10.604941,3.807180,10.557676,12.040169,11.045359,9.950208,11.001129,9.945029,11.796288
TCGA-18-3411-01A,9.862795,8.375418,8.488004,9.268918,9.906991,11.893865,8.022793,13.083424,13.635985,8.943100,...,7.853350,10.326932,9.302395,8.090365,11.968898,9.097285,6.893264,8.641221,7.371018,14.392423
TCGA-18-3412-01A,9.347034,6.788506,8.340568,8.590906,10.489859,11.563028,7.324029,12.494692,12.583905,9.294969,...,8.316195,8.718400,7.291973,9.008665,11.798960,9.520752,8.549325,8.331835,6.550331,15.195619
TCGA-18-3414-01A,9.125424,10.198092,8.646927,8.697841,11.691098,11.769628,8.650908,11.870946,12.437843,10.494982,...,11.530794,10.137898,13.872718,9.448742,12.617170,9.533121,8.446231,8.855642,6.974900,13.180301
TCGA-18-3415-01A,8.222495,9.376307,8.730298,8.759455,10.931066,12.430472,7.493382,12.026757,13.243918,9.560973,...,9.325884,9.180527,10.507136,9.071447,11.997483,9.310849,9.426858,8.637023,7.224531,15.274533
TCGA-18-3416-01A,8.716105,7.093497,8.948906,8.012303,10.136643,10.563644,7.971197,12.991445,13.239963,10.246069,...,8.594673,9.663379,2.449746,8.067352,12.130306,10.019007,6.989657,10.110671,9.367282,14.333194


In [133]:
from sklearn.cluster import KMeans


In [134]:
kmeans = KMeans(n_clusters=4)

In [135]:
df = centroids.values
centroid_values = (df.T - df.T.mean()) / (df.T.max() - df.T.min()).T
kmeans.cluster_centers_ = centroid_values.T

In [142]:
kmeans.cluster_centers_.shape

(4, 196)

In [137]:
kmeans.predict(df_norm)

array([3, 1, 1, 3, 0, 1, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 3, 1, 3, 2, 1,
       3, 2, 0, 3, 2, 1, 1, 1, 1, 3, 3, 2, 1, 1, 2, 3, 2, 1, 1, 1, 3, 2, 1,
       0, 0, 2, 1, 2, 0, 2, 1, 3, 2, 1, 2, 3, 1, 1, 3, 1, 3, 3, 1, 1, 1, 0,
       2, 3, 3, 2, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 3, 1, 1, 3, 1, 3, 1, 3,
       1, 3, 1, 1, 2, 3, 2, 3, 1, 3, 3, 3, 2, 0, 0, 0, 1, 0, 0, 0, 1, 3, 1,
       2, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 0, 3, 1, 1, 3, 0, 2, 2, 0, 2, 1, 2,
       3, 0, 3, 2, 1, 2, 3, 2, 1, 3, 2, 1, 2, 0, 1, 2, 0, 2, 3, 3, 1, 1, 1,
       3, 3, 1, 3, 3, 1, 0, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 3, 1, 1, 2, 1, 3,
       1, 3, 0, 1, 3, 2, 1, 1, 2, 3, 2, 1, 2, 1, 2, 3, 2, 3, 1, 2, 1, 2, 0,
       2, 3, 2, 3, 1, 0, 1, 3, 2, 3, 1, 3, 2, 2, 3, 1, 3, 3, 1, 3, 3, 1, 1,
       3, 1, 1, 3, 1, 3, 2, 1, 0, 3, 1, 1, 1, 3, 1, 1, 1, 1, 0, 3, 2, 1, 3,
       3, 0, 3, 3, 1, 2, 1, 1, 3, 1, 2, 3, 1, 1, 3, 1, 1, 1, 3, 3, 1, 1, 1,
       2, 3,

In [140]:
len(kmeans.predict(df_norm))

552

In [139]:
pd.Series(kmeans.predict(df_norm)).value_counts()

1    249
3    189
2     72
0     42
dtype: int64

In [ ]:
primitive 7%, classical 45%, secretory 13%, basal 34%